The most popular GNN models can be written as follows:

$$
h_i^{(l+1)}=\sigma(b^{(l)}+\sum_{j\in\mathcal{N}(i)}e_{ij}h_j^{(l)}W^{(l)})
$$

where $h_i^{(l+1)}$ is the output feature, $\sigma$ is the activation function, $e_{ij}$ is the edge weight, $W^{(l)}$ is the learnable parameters(可学习参数), $b^{(l)}$ is the bias.

First, you will use the PyTorch-Geometric(PyG) to implement this convolution layer.
首先，您将使用PyTorch-Geometric(PyG)来实现这个卷积层。

In [10]:
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing

class PyG_conv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(PyG_conv, self).__init__(aggr='add')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.W = nn.Parameter(torch.ones((in_channels, out_channels)))
        self.b = nn.Parameter(torch.ones(out_channels))

    def forward(self, x, edge_index, edge_weight):
        return self.propagate(edge_index, x=x, edge_weight=edge_weight)

    def message(self, x_j, edge_weight):
        return edge_weight.view(-1, 1) * x_j

    def update(self, aggr_out):
        return torch.matmul(aggr_out, self.W) + self.b

You may run the following code to check the correctness.

In [11]:
import numpy as np
edge_index = torch.tensor([[0,1,1,2,2,4],[2,0,2,3,4,3]])
# h0
x = torch.ones((5, 8))
edge_weight = 2 * torch.ones(6)
conv = PyG_conv(8, 4)
output = conv(x, edge_index, edge_weight)
assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                    [ 1.,  1.,  1.,  1.],
                    [33., 33., 33., 33.],
                    [33., 33., 33., 33.],
                    [17., 17., 17., 17.]])

Now, you will implement the same functions with DGL.

The most popular GNN models can be written as follows:

$$
h_i^{(l+1)}=\sigma(b^{(l)}+\sum_{j\in\mathcal{N}(i)}e_{ij}h_j^{(l)}W^{(l)})
$$

where $h_i^{(l+1)}$ is the output feature, $\sigma$ is the activation function(激活函数), $e_{ij}$ is the edge weight, $W^{(l)}$ is the learnable parameters(可学习参数), $b^{(l)}$ is the bias.

First, you will use the PyTorch-Geometric(PyG) to implement this convolution layer.
首先，您将使用PyTorch-Geometric(PyG)来实现这个卷积层。

Also, you can also run the code below to check the correctness.

In [8]:
import dgl.function as fn
import dgl

class DGL_conv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(DGL_conv, self).__init__()
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.W = nn.Parameter(torch.ones(in_channel, out_channel))
        self.b = nn.Parameter(torch.ones(out_channel))

    def forward(self, g, h, edge_weight):
        g.ndata['h'] = h
        g.edata['weight'] = edge_weight
        g.update_all(message_func=fn.u_mul_e('h', 'weight', 'm'), reduce_func=fn.sum('m', 'h_neigh'))
        h_neigh = g.ndata['h_neigh']
        return torch.matmul(h_neigh, self.W) + self.b

In [9]:
src = torch.tensor([0, 1, 1, 2, 2, 4])
dst = torch.tensor([2, 0, 2, 3, 4, 3])
h = torch.ones((5, 8))
g = dgl.graph((src, dst))
edge_weight = 2 * torch.ones(6)
conv = DGL_conv(8, 4)
output = conv(g, h, edge_weight)
import numpy as np
assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                    [ 1.,  1.,  1.,  1.],
                    [33., 33., 33., 33.],
                    [33., 33., 33., 33.],
                    [17., 17., 17., 17.]])